In [4]:
from subprocess import check_output
path = "../data/data-science-london-scikit-learn"
print(check_output(["ls", path]).decode("utf8"))

test.csv
train.csv
trainLabels.csv



In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

train = pd.read_csv(os.path.join(path, 'train.csv'), header=None)
trainLabels = pd.read_csv(os.path.join(path, 'trainLabels.csv'), header=None)
test = pd.read_csv(os.path.join(path, 'test.csv'), header=None)
print(f"X:{train.shape}Y:{trainLabels.shape} Test:{test.shape}")

X:(1000, 40)Y:(1000, 1) Test:(9000, 40)


In [6]:
train.head(3)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.850465,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436


In [7]:
trainLabels.head(3)

,0
0,1
1,0
2,0


In [8]:
test.head(3)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,2.808909,-0.242894,-0.546421,0.255162,1.749736,-0.030458,-1.322071,3.578071,-0.667578,-0.884257,...,-0.261688,-0.224375,-1.675606,-0.479584,-0.244388,-0.672355,0.517860,0.010665,-0.419214,2.818387
1,-0.374101,0.537669,0.081063,0.756773,0.915231,2.557282,3.703187,1.673835,-0.764122,-1.228040,...,-0.969463,0.574154,-2.200519,-1.612240,0.179031,-2.924596,0.643610,-1.470939,-0.067408,-0.976265
2,-0.088370,0.154743,0.380716,-1.176126,1.699867,-0.258627,-1.384999,1.093584,1.596633,0.230631,...,-0.769885,-0.005143,1.467490,0.483803,-3.542981,0.814561,-1.652948,1.265866,-1.749248,1.773784


In [9]:
# the eda show we doesn't need normalization

In [15]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, np.ravel(trainLabels), test_size=0.1)

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=7)

In [16]:
def findBestGradientBoosting():
    # Choose some parameter combinations to try
    param_grid = {'n_estimators': [ 200, 300, 400], 
                  'max_depth': [2, 3, 5], 
                  'learning_rate': [0.01,0.05,0.07, 0.08]
                 }

    # 建立模型
    clf = GradientBoostingClassifier(random_state=7)
    # Run the grid search
    grid_obj = GridSearchCV(estimator = clf,param_grid = param_grid, verbose=1)
    grid_obj = grid_obj.fit(X_train, y_train)

    # 印出最佳結果與最佳參數
    print("Best Accuracy: %f using %s" % (grid_obj.best_score_, grid_obj.best_params_))
    return grid_obj.best_estimator_

In [17]:
# 使用最佳化參數測試
clf = findBestGradientBoosting()
clf.fit(X_train, y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed:  3.8min finished


Best Accuracy: 0.872222 using {'learning_rate': 0.07, 'max_depth': 5, 'n_estimators': 300}


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.07, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='auto',
                           random_state=7, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [18]:
predictions = clf.predict(X_test)

In [19]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, predictions)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.9591836734693878

In [20]:
result = clf.predict(test)

In [21]:
submission = pd.DataFrame(result)
print(submission.shape)

(9000, 1)


In [22]:
submission.columns = ['Solution']
submission['Id'] = np.arange(1,submission.shape[0]+1)
submission = submission[['Id', 'Solution']]
submission.to_csv('submission.csv',index=False)